In [4]:
!pip install onnxruntime 

  Using cached onnxruntime-1.14.1-cp39-cp39-win_amd64.whl (6.5 MB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)


In [1]:
import cv2
import os
import numpy as np
import pytesseract as pt
import onnxruntime as ort
import time
from concurrent.futures import ThreadPoolExecutor

pt.pytesseract.tesseract_cmd=r"C:\Program Files\Tesseract-OCR\tesseract.exe"
INPUT_WIDTH = 640
INPUT_HEIGHT = 640

#os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;tcp"

ort_session = ort.InferenceSession('best.onnx')
print("Model Loaded", ort_session)

net = ort_session

   # Function getdetection
    
def get_detections(img, ort_session):
    image = img.copy()
    row, col, d = image.shape

    max_rc = max(row, col)
    input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
    input_image[0:row, 0:col] = image

    input_image_resized = cv2.resize(input_image, (INPUT_WIDTH, INPUT_HEIGHT))
    input_image_resized = input_image_resized.transpose(2, 0, 1)
    input_image_resized = input_image_resized.reshape(1, 3, INPUT_WIDTH, INPUT_HEIGHT).astype('float32')
    input_image_resized /= 255.0

    ort_inputs = {ort_session.get_inputs()[0].name: input_image_resized}
    ort_outs = ort_session.run(None, ort_inputs)
    detections = ort_outs[0][0]

    return input_image, detections

   # function non max suression

def non_maximum_suppression(input_image, detections):
    boxes = []
    confidences = []

    image_w, image_h = input_image.shape[:2]
    x_factor = image_w / INPUT_WIDTH
    y_factor = image_h / INPUT_HEIGHT

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0.6:
            class_score = row[5]
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]

                left = int((cx - 0.5 * w) * x_factor)
                top = int((cy - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])

                confidences.append(confidence)
                boxes.append(box)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()
    try:
        index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.55, 0.75).flatten()
    except:
        index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.55, 0.75)
    return boxes_np, confidences_np, index

  # function text extraction

def extract_text(image, bbox):
    x, y, w, h = bbox
    roi = image[y:y+h, x:x+w]

    if 0 in roi.shape:
        return ''
    else:
        text = pt.image_to_string(roi, lang='eng', config='--psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
        text = text.strip()
        print("Text of plate is: ", text)
        return text

 # functioon drawing   

def drawings(image, boxes_np, confidences_np, index):
    for ind in index:
        x, y, w, h = boxes_np[ind]
        bb_conf = confidences_np[ind]
        conf_text = 'plate: {:.0f}%'.format(bb_conf * 100)
        license_text = extract_text(image, boxes_np[ind])

        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 255), 2)
        cv2.rectangle(image, (x, y - 30), (x + w, y), (255, 0, 255), -1)
        cv2.rectangle(image, (x, y + h), (x + w, y + h + 30), (0, 0, 0), -1)

        cv2.putText(image, conf_text, (x, y - 13), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(image, license_text, (x, y + h + 27), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return image
    
    
    
def yolo_predictions(img, ort_session):
    input_image, detections = get_detections(img, ort_session)
    boxes_np, confidences_np, index = non_maximum_suppression(input_image, detections)
    result_img = drawings(img, boxes_np, confidences_np, index)
    return result_img

def process_frame(frame):
    frame = cv2.resize(frame, (600, 400))
    result = yolo_predictions(frame, net)
    return result


executor = ThreadPoolExecutor(max_workers=3)  # Adjust max_workers based on your CPU core count

cap = cv2.VideoCapture("last_video.mp4")

print("capture device is open: " + str(cap.isOpened()))
fps_start_time = time.time()
fps = 0

frame_futures = []

while True:
    success, frame = cap.read()
    if not success:
        break

    frame_future = executor.submit(process_frame, frame)
    frame_futures.append(frame_future)
    
    # Process the oldest frame
    if len(frame_futures) > 4:  # Adjust the buffer size based on your CPU core count
        result = frame_futures.pop(0).result()
        fps_end_time = time.time()
        fps_diff_time = fps_end_time - fps_start_time
        fps = 1 / fps_diff_time
        fps_start_time = fps_end_time
        fps_text = "FPS:{:.2f}".format(fps)
        cv2.putText(result, fps_text, (5, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255), 1)
        cv2.imshow("webcam", result)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Clean up
executor.shutdown(wait=True)



Model Loaded <onnxruntime.capi.onnxruntime_inference_collection.InferenceSession object at 0x000002109500A220>
capture device is open: True
Text of plate is: Text of plate is:  
 1
Text of plate is:  
Text of plate is:  
Text of plate is:  
Text of plate is:  
Text of plate is:  
Text of plate is:  
Text of plate is:  
Text of plate is:  
Text of plate is:  
Text of plate is:  4
Text of plate is:  
Text of plate is:  4
Text of plate is:  4
Text of plate is:  
Text of plate is:  
Text of plate is:  F


ZeroDivisionError: float division by zero

Text of plate is:  
Text of plate is:  G
Text of plate is:  
Text of plate is:  


In [2]:
import cv2
import os
import numpy as np
import pytesseract as pt
import onnxruntime as ort
import time
from concurrent.futures import ThreadPoolExecutor

#os.chdir("/Users/PranavGupta/Downloads/plate_copy")

INPUT_WIDTH = 640
INPUT_HEIGHT = 640

os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;tcp"

ort_session = ort.InferenceSession('best.onnx')
print("Model Loaded", ort_session)
"""
net = ort_session

def get_detections(img, ort_session):
    image = img.copy()
    row, col, d = image.shape

    max_rc = max(row, col)
    input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
    input_image[0:row, 0:col] = image

    input_image_resized = cv2.resize(input_image, (INPUT_WIDTH, INPUT_HEIGHT))
    input_image_resized = input_image_resized.transpose(2, 0, 1)
    input_image_resized = input_image_resized.reshape(1, 3, INPUT_WIDTH, INPUT_HEIGHT).astype('float32')
    input_image_resized /= 255.0

    ort_inputs = {ort_session.get_inputs()[0].name: input_image_resized}
    ort_outs = ort_session.run(None, ort_inputs)
    detections = ort_outs[0][0]

    return input_image, detections

def non_maximum_suppression(input_image, detections):
    boxes = []
    confidences = []

    image_w, image_h = input_image.shape[:2]
    x_factor = image_w / INPUT_WIDTH
    y_factor = image_h / INPUT_HEIGHT

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0.6:
            class_score = row[5]
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]

                left = int((cx - 0.5 * w) * x_factor)
                top = int((cy - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])

                confidences.append(confidence)
                boxes.append(box)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()
    try:
        index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.55, 0.75).flatten()
    except:
        index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.55, 0.75)
    return boxes_np, confidences_np, index

def extract_text(image, bbox):
    x, y, w, h = bbox
    roi = image[y:y+h, x:x+w]

    if 0 in roi.shape:
        return ''
    else:
        text = pt.image_to_string(roi, lang='eng', config='--psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
        text = text.strip()
        print("Text of plate is: ", text)
        return text

def drawings(image, boxes_np, confidences_np, index):
    for ind in index:
        x, y, w, h = boxes_np[ind]
        bb_conf = confidences_np[ind]
        conf_text = 'plate: {:.0f}%'.format(bb_conf * 100)
        license_text = extract_text(image, boxes_np[ind])

        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 255), 2)
        cv2.rectangle(image, (x, y - 30), (x + w, y), (255, 0, 255), -1)
        cv2.rectangle(image, (x, y + h), (x + w, y + h + 30), (0, 0, 0), -1)

        cv2.putText(image, conf_text, (x, y - 13), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(image, license_text, (x, y + h + 27), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return image

def yolo_predictions(img, ort_session):
    input_image, detections = get_detections(img, ort_session)
    boxes_np, confidences_np, index = non_maximum_suppression(input_image, detections)
    result_img = drawings(img, boxes_np, confidences_np, index)
    return result_img

def process_frame(frame):
    frame = cv2.resize(frame, (600, 400))
    result = yolo_predictions(frame, net)
    return result

executor = ThreadPoolExecutor(max_workers=3)  # Adjust max_workers based on your CPU core count

cap = cv2.VideoCapture("last_video.mp4")

print("capture device is open: " + str(cap.isOpened()))
fps_start_time = time.time()
fps = 0

frame_futures = []

while True:
    success, frame = cap.read()
    if not success:
        break

    frame_future = executor.submit(process_frame, frame)
    frame_futures.append(frame_future)

    # Process the oldest frame
    if len(frame_futures) > 4:  # Adjust the buffer size based on your CPU core count
        result = frame_futures.pop(0).result()
        fps_end_time = time.time()
        fps_diff_time = fps_end_time - fps_start_time
        fps = 1 / fps_diff_time
        fps_start_time = fps_end_time
        fps_text = "FPS:{:.2f}".format(fps)
        cv2.putText(result, fps_text, (5, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255), 1)
        cv2.imshow("webcam", result)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Clean up
executor.shutdown(wait=True)
"""

ModuleNotFoundError: No module named 'onnxruntime'